In [1]:
import pyupbit as ub
import ccxt
import pandas as pd
import datetime
import time
import json

In [3]:
def get_crypto_data(tickers, interval="day", count=3000):
    """
    tickers: 암호화폐의 ticker 목록 (예: ["KRW-BTC", "KRW-ETH"])
    interval: 데이터 간격 ("minute1", "minute3", "minute5", "minute10", "minute15", "minute30", "minute60", "minute240", "day", "week", "month")
    count: 가져올 데이터의 갯수 (최대 200)
    
    return: ticker를 key로 하고, 해당 데이터를 value로 하는 dictionary 반환
    """
    data = {}
    for ticker in tickers:
        ohlcv = ub.get_ohlcv(ticker, interval=interval, count=count)
        #get_daily_ohlcv_from_base(ticker=ticker, base=09))
        data[ticker] = ohlcv
    return data

# 예시 사용법
tickers = ub.get_tickers(fiat="KRW")

crypto_data = get_crypto_data(tickers)
for ticker, ohlcv in crypto_data.items():
    print(ticker)
    print(ohlcv.head())  # 상위 5개 데이터 출력

KRW-BTC
                          open       high        low      close      volume  \
2017-09-25 09:00:00  4201000.0  4333000.0  4175000.0  4322000.0  132.484755   
2017-09-26 09:00:00  4317000.0  4418000.0  4311000.0  4321000.0   22.788340   
2017-09-27 09:00:00  4322000.0  4677000.0  4318000.0  4657000.0   32.269662   
2017-09-28 09:00:00  4657000.0  4772000.0  4519000.0  4586000.0   80.588243   
2017-09-29 09:00:00  4586000.0  4709000.0  4476000.0  4657000.0   59.352373   

                            value  
2017-09-25 09:00:00  5.602146e+08  
2017-09-26 09:00:00  9.950724e+07  
2017-09-27 09:00:00  1.448276e+08  
2017-09-28 09:00:00  3.721860e+08  
2017-09-29 09:00:00  2.724558e+08  
KRW-ETH
                         open      high       low     close       volume  \
2017-09-25 09:00:00  324500.0  326500.0  320000.0  325000.0  1421.855427   
2017-09-26 09:00:00  325000.0  328500.0  320000.0  321500.0     0.051600   
2017-09-27 09:00:00  321000.0  342500.0  320000.0  342500.0     0

In [5]:
len(crypto_data)

118

In [6]:
def add_index_as_date_column(data):
    """
    data: 입력 딕셔너리. 이 딕셔너리의 각 값은 DataFrame이어야 함.
    return: 각 DataFrame에 인덱스를 "date"라는 새로운 열로 추가한 딕셔너리.
    """
    for key, value in data.items():
        if isinstance(value, pd.DataFrame):
            value['date'] = value.index
            data[key] = value
    return data

In [7]:
result_data = add_index_as_date_column(crypto_data)
for key, value in result_data.items():
    print(key)
    print(value)

KRW-BTC
                           open        high         low       close  \
2017-09-25 09:00:00   4201000.0   4333000.0   4175000.0   4322000.0   
2017-09-26 09:00:00   4317000.0   4418000.0   4311000.0   4321000.0   
2017-09-27 09:00:00   4322000.0   4677000.0   4318000.0   4657000.0   
2017-09-28 09:00:00   4657000.0   4772000.0   4519000.0   4586000.0   
2017-09-29 09:00:00   4586000.0   4709000.0   4476000.0   4657000.0   
...                         ...         ...         ...         ...   
2023-09-17 09:00:00  35939000.0  35996000.0  35650000.0  35758000.0   
2023-09-18 09:00:00  35758000.0  36801000.0  35658000.0  36000000.0   
2023-09-19 09:00:00  36000000.0  36820000.0  35750000.0  36500000.0   
2023-09-20 09:00:00  36509000.0  36743000.0  36150000.0  36501000.0   
2023-09-21 09:00:00  36502000.0  36659000.0  36150000.0  36191000.0   

                          volume         value                date  
2017-09-25 09:00:00   132.484755  5.602146e+08 2017-09-25 09:00:00  
2

In [9]:
def dict_to_dataframe(data, key_column_name="key"):
    """
    data: 변환하려는 딕셔너리. 이 딕셔너리의 각 값은 DataFrame이어야 함.
    key_column_name: 딕셔너리의 키 값을 저장할 열의 이름.
    
    return: 딕셔너리를 변환한 DataFrame. 딕셔너리의 각 키 값은 새로운 열로 추가됨.
    """
    dfs = []
    for key, value in data.items():
        if isinstance(value, pd.DataFrame):
            df = value.copy()
            df[key_column_name] = key
            dfs.append(df)
    
    return pd.concat(dfs, ignore_index=True)

In [10]:
result_df = dict_to_dataframe(result_data, key_column_name="currency")
print(result_df)

             open       high        low      close        volume  \
0       4201000.0  4333000.0  4175000.0  4322000.0  1.324848e+02   
1       4317000.0  4418000.0  4311000.0  4321000.0  2.278834e+01   
2       4322000.0  4677000.0  4318000.0  4657000.0  3.226966e+01   
3       4657000.0  4772000.0  4519000.0  4586000.0  8.058824e+01   
4       4586000.0  4709000.0  4476000.0  4657000.0  5.935237e+01   
...           ...        ...        ...        ...           ...   
168694      190.0      191.0      165.0      166.0  3.286900e+08   
168695      165.0      172.0      161.0      167.0  1.633393e+08   
168696      167.0      170.0      162.0      168.0  1.079979e+08   
168697      168.0      175.0      164.0      167.0  1.886087e+08   
168698      167.0      177.0      165.0      167.0  1.835145e+08   

               value                date currency  
0       5.602146e+08 2017-09-25 09:00:00  KRW-BTC  
1       9.950724e+07 2017-09-26 09:00:00  KRW-BTC  
2       1.448276e+08 2017-0

In [11]:
result_df = result_df[['date', 'open', 'high', 'low', 'close', 'volume', 'value', 'currency']]
result_df.to_csv("KRW_cryptos_krw_pyupbit.csv", index=False)

,date,open,high,low,close,volume,value,currency
0,2017-09-25 09:00:00,4201000.0,4333000.0,4175000.0,4322000.0,1.324848e+02,5.602146e+08,KRW-BTC
1,2017-09-26 09:00:00,4317000.0,4418000.0,4311000.0,4321000.0,2.278834e+01,9.950724e+07,KRW-BTC
2,2017-09-27 09:00:00,4322000.0,4677000.0,4318000.0,4657000.0,3.226966e+01,1.448276e+08,KRW-BTC
3,2017-09-28 09:00:00,4657000.0,4772000.0,4519000.0,4586000.0,8.058824e+01,3.721860e+08,KRW-BTC
4,2017-09-29 09:00:00,4586000.0,4709000.0,4476000.0,4657000.0,5.935237e+01,2.724558e+08,KRW-BTC
...,...,...,...,...,...,...,...,...
168694,2023-09-17 09:00:00,190.0,191.0,165.0,166.0,3.286900e+08,5.776295e+10,KRW-SEI
168695,2023-09-18 09:00:00,165.0,172.0,161.0,167.0,1.633393e+08,2.709218e+10,KRW-SEI
168696,2023-09-19 09:00:00,167.0,170.0,162.0,168.0,1.079979e+08,1.795616e+10,KRW-SEI
168697,2023-09-20 09:00:00,168.0,175.0,164.0,167.0,1.886087e+08,3.190771e+10,KRW-SEI


In [21]:
# def save_to_json(data, filename):
#     """
#     data: 저장하려는 딕셔너리 데이터. 이 데이터 안에 DataFrame 객체가 포함될 수 있음.
#     filename: 저장하려는 파일의 이름 (예: "data.json")
#     """
#     # DataFrame 객체를 JSON 문자열로 변환
#     serialized_data = {}
#     for key, value in data.items():
#         if isinstance(value, pd.DataFrame):
#             serialized_data[key] = value.to_json(orient='split')
#         else:
#             serialized_data[key] = value
            
#     with open(filename, 'w', encoding='utf-8') as f:
#         json.dump(serialized_data, f, ensure_ascii=False, indent=4)

In [22]:
# save_to_json(crypto_data, "crypto_data.json")

In [4]:
# def load_from_json(filename):
#     """
#     filename: 불러오려는 파일의 이름 (예: "data.json")
    
#     return: JSON 파일의 내용을 딕셔너리 형태로 반환. 내부에 DataFrame 문자열이 있을 경우 DataFrame으로 변환하여 반환.
#     """
#     with open(filename, 'r', encoding='utf-8') as f:
#         data = json.load(f)
    
#     # JSON 문자열을 DataFrame으로 변환
#     for key, value in data.items():
#         if isinstance(value, str) and "index" in value and "data" in value and "columns" in value:
#             data[key] = pd.read_json(value, orient='split')
    
#     return data


In [24]:
# loaded_data = load_from_json("crypto_data.json")
# for key, value in loaded_data.items():
#     print(key)
#     if isinstance(value, pd.DataFrame):
#         print(value.head())  # DataFrame의 경우 상위 5개 데이터 출력
#     else:
#         print(value)

KRW-BTC
                         open      high       low     close        volume  \
2018-03-02 09:00:00  12214000  12581000  12123000  12465000  23945.117557   
2018-03-03 09:00:00  12465000  12831000  12347000  12831000  18163.175191   
2018-03-04 09:00:00  12831000  12840000  12255000  12592000  16125.743255   
2018-03-05 09:00:00  12592000  12899000  12451000  12746000  13919.500362   
2018-03-06 09:00:00  12745000  12746000  11900000  12023000  15007.199784   

                            value  
2018-03-02 09:00:00  2.956371e+11  
2018-03-03 09:00:00  2.289656e+11  
2018-03-04 09:00:00  2.013402e+11  
2018-03-05 09:00:00  1.757026e+11  
2018-03-06 09:00:00  1.853153e+11  
KRW-ETH
                       open    high     low   close        volume  \
2018-03-02 09:00:00  973900  979800  959700  965000  17451.831318   
2018-03-03 09:00:00  965000  968500  951100  956300  15285.995637   
2018-03-04 09:00:00  956500  958000  927300  947900  15385.445487   
2018-03-05 09:00:00  947600  

In [22]:
# def add_index_as_date_column(data):
#     """
#     data: 입력 딕셔너리. 이 딕셔너리의 각 값은 DataFrame이어야 함.
#     return: 각 DataFrame에 인덱스를 "date"라는 새로운 열로 추가한 딕셔너리.
#     """
#     for key, value in data.items():
#         if isinstance(value, pd.DataFrame):
#             value['date'] = value.index
#             data[key] = value
#     return data

In [23]:
# result_data = add_index_as_date_column(crypto_data)
# for key, value in result_data.items():
#     print(key)
#     print(value)

KRW-BTC
                           open        high         low       close  \
2017-09-25 09:00:00   4201000.0   4333000.0   4175000.0   4322000.0   
2017-09-26 09:00:00   4317000.0   4418000.0   4311000.0   4321000.0   
2017-09-27 09:00:00   4322000.0   4677000.0   4318000.0   4657000.0   
2017-09-28 09:00:00   4657000.0   4772000.0   4519000.0   4586000.0   
2017-09-29 09:00:00   4586000.0   4709000.0   4476000.0   4657000.0   
...                         ...         ...         ...         ...   
2023-09-16 09:00:00  35907000.0  36051000.0  35800000.0  35938000.0   
2023-09-17 09:00:00  35939000.0  35996000.0  35650000.0  35758000.0   
2023-09-18 09:00:00  35758000.0  36801000.0  35658000.0  36000000.0   
2023-09-19 09:00:00  36000000.0  36820000.0  35750000.0  36500000.0   
2023-09-20 09:00:00  36509000.0  36743000.0  36319000.0  36480000.0   

                          volume         value                date  
2017-09-25 09:00:00   132.484755  5.602146e+08 2017-09-25 09:00:00  
2

In [25]:
# def dict_to_dataframe(data, key_column_name="key"):
#     """
#     data: 변환하려는 딕셔너리. 이 딕셔너리의 각 값은 DataFrame이어야 함.
#     key_column_name: 딕셔너리의 키 값을 저장할 열의 이름.
    
#     return: 딕셔너리를 변환한 DataFrame. 딕셔너리의 각 키 값은 새로운 열로 추가됨.
#     """
#     dfs = []
#     for key, value in data.items():
#         if isinstance(value, pd.DataFrame):
#             df = value.copy()
#             df[key_column_name] = key
#             dfs.append(df)
    
#     return pd.concat(dfs, ignore_index=True)

In [26]:
# result_df = dict_to_dataframe(result_data, key_column_name="currency")
# print(result_df)

             open       high        low      close        volume  \
0       4201000.0  4333000.0  4175000.0  4322000.0  1.324848e+02   
1       4317000.0  4418000.0  4311000.0  4321000.0  2.278834e+01   
2       4322000.0  4677000.0  4318000.0  4657000.0  3.226966e+01   
3       4657000.0  4772000.0  4519000.0  4586000.0  8.058824e+01   
4       4586000.0  4709000.0  4476000.0  4657000.0  5.935237e+01   
...           ...        ...        ...        ...           ...   
168576      168.0      202.0      164.0      190.0  9.078253e+08   
168577      190.0      191.0      165.0      166.0  3.286900e+08   
168578      165.0      172.0      161.0      167.0  1.633393e+08   
168579      167.0      170.0      162.0      168.0  1.079979e+08   
168580      168.0      175.0      167.0      171.0  8.458786e+07   

               value                date currency  
0       5.602146e+08 2017-09-25 09:00:00  KRW-BTC  
1       9.950724e+07 2017-09-26 09:00:00  KRW-BTC  
2       1.448276e+08 2017-0